<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/notebooks/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.getcwd()
os.chdir('/content/drive/My Drive/Partisan-Responses-master')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/Partisan-Responses-master'

In [ ]:
!pip install allennlp hnswlib spacy allennlp-models

In [ ]:
import pandas as pd
import spacy
from Answer import Answer
import utils
import re
import pickle

identifier='second_test'
last_check=1
step_size=100

df = pd.read_pickle("final_data.pkl")
def parse_entry(entry,verb_dict,verb_list):
    result=dict()
    result['question']=' '.join([token.text for token in utils.sp(entry['question'])])
    phrase_corpus, triplet_id, parsed_text,parsed=Answer(entry['answer']).create_training(verb_dict,verb_list)
    result['corpus']=' ; '.join(phrase_corpus)
    result['tags']=' '.join(['<phrase>']*len(phrase_corpus))
    result['triplet_id']=' ; '.join([re.sub('\,','',str(x))[1:-1] for x in triplet_id])
    result['parsed_text']=parsed_text
    result['parsed']=' '.join([str(x) for x in parsed])
    return result

def load_pickles(identifier, checkpoint):
  if os.path.exists("verb_dict"+identifier+str(checkpoint)+".pickle"):
    with open('verb_dict'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      verb_dict = pickle.load(handle)
    with open('verb_list'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      verb_list = pickle.load(handle)
    with open('result'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      result = pickle.load(handle)
  else:
    print('File does not exist yet.')
    verb_dict=dict()
    verb_list=[]
    result=[]
  return verb_dict, verb_list, result

def dump_pickles(identifier, checkpoint, verb_dict, verb_list, result):
  with open("verb_dict"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(verb_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open("verb_list"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(verb_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open("result"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)
  return 

verb_dict, verb_list, result=load_pickles(identifier, last_check)
print(verb_dict)
print(verb_list)
print(result)
for i in range(last_check+1,int(df.shape[0]/step_size)):
  print(i)
  df[step_size*(i-1):step_size*i].apply(lambda x: result.append(parse_entry(x,verb_dict,verb_list)), axis=1)
  dump_pickles(identifier,i,verb_dict,verb_list,result)
#df[:5].apply(lambda x: result.append(parse_entry(x,verb_dict,verb_list)), axis=1
print('done with ordered')
df[step_size*i:].apply(lambda x: result.append(parse_entry(x,verb_dict,verb_list)), axis=1)
df=pd.DataFrame(result) 
df=df.sample(frac=1, random_state=36)
df=df.dropna(axis=0, how='any')
df=df[df.iloc[:,4].apply(lambda x: len(x)<1000)]

m=df.iloc[:50]
m.to_csv(identifier+'.control.tsv', sep='\t', index=False, header=False)
m=df.iloc[50:1050]
m.to_csv(identifier+'.val.tsv', sep='\t', index=False, header=False)
m=df.iloc[1050:2050]
m.to_csv(identifier+'.test.tsv', sep='\t', index=False, header=False)
m=df.iloc[2050:]
m.to_csv(identifier+'.train.tsv', sep='\t', index=False, header=False)

with open(identifier+'.vocab', 'w') as filehandle:
    filehandle.writelines("%s\n" % verb.upper() for verb in verb_list)

INFO:absl:Using /tmp/tfhub_modules to cache modules.


{'employ': 0, 'see': 1, 'make': 2, 'elect': 3, 'require': 4, 'decide': 5, 'punish': 6, 'give': 7, 'carry': 8, 'think': 9, 'overrule': 10, 'bind': 11, 'mai': 12, 'touch': 13, 'imterrapted': 14, 'say': 15, 'liviigwould': 16, 'violate': 17, 'defy': 18, 'coie': 19, 'sailing': 20, 'know': 21, 'reading': 22, 'read': 23, 'allow': 24, 'wouldhave': 25, 'expect': 26, 'bring': 27, 'remove': 28, 'place': 29, 'flaunted': 30, 'question': 31, 'deal': 32, 'bury': 33, 'ashaine': 34, 'thought': 35, 'brand': 36, 'concived': 37, 'believe': 38, 'mu': 39, 'consider': 40, 'pay': 41, 'propose': 42, 'provide': 43, 'devolve': 44, 'appropriate': 45, 'issue': 46, 'accomplish': 47, 'buy': 48, 'further': 49, 'take': 50, 'send': 51, 'bluecoate': 52, 'perform': 53, 'mail': 54, 'fight': 55, 'recognize': 56, 'oneof': 57, 'look': 58, 'exchange': 59, 'hold': 60, '1e': 61, 'differ': 62, 'travel': 63, 'produce': 64, 'go': 65, 'depend': 66, 'resume': 67, 'adjust': 68, 'fund': 69, 'turn': 70, 'start': 71, 'show': 72, 'excuse

/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
